In [8]:
%%capture
!pip install advertools

In [11]:
import pandas as pd
pd.options.display.max_columns = None
import re
import os
import time
from tqdm import tqdm


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Sample lines from the log file

In [13]:
# There is a minor bug in this regex, it misses the last field. I'll fix this soon. 

common_regex = '^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-)'
combined_regex = '^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-) "(?P<referrer>[^"]*)" "(?P<useragent>[^"]*)'
columns = ['client', 'userid', 'datetime', 'method', 'request', 'status', 'size', 'referer', 'user_agent']

# The `logs_to_df` function

In [14]:
import time
import re
import pandas as pd


def logs_to_df(logfile, output_dir, errors_file):
    with open(logfile) as source_file:
        linenumber = 0
        parsed_lines = []
        for line in tqdm(source_file):
            try:
                log_line = re.findall(combined_regex, line)[0]
                parsed_lines.append(log_line)
            except Exception as e:
                with open(errors_file, 'at') as errfile:
                    print((line, str(e)), file=errfile)
                continue
            linenumber += 1
            if linenumber % 250_000 == 0:
                df = pd.DataFrame(parsed_lines, columns=columns)
                df.to_parquet(f'{output_dir}/file_{linenumber}.parquet')
                parsed_lines.clear()
        else:
            df = pd.DataFrame(parsed_lines, columns=columns)
            df.to_parquet(f'{output_dir}/file_{linenumber}.parquet')
            parsed_lines.clear()

In [15]:
%time logs_to_df(logfile='/content/drive/MyDrive/Colab Notebooks/big_data/access.log', output_dir='/content/drive/MyDrive/Colab Notebooks/big_data/paquet_dir/', errors_file='errors.txt')

10365152it [01:38, 104952.42it/s]


CPU times: user 1min 18s, sys: 7.63 s, total: 1min 26s
Wall time: 1min 39s


In [16]:
%time logs_df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/big_data/paquet_dir/')

CPU times: user 12 s, sys: 2.99 s, total: 15 s
Wall time: 11 s


Reading the whole directory takes about nine seconds. We now check the size of the resulting directory on disk:

In [19]:
logs_df

,client,userid,datetime,method,request,status,size,referer,user_agent
0,37.152.163.59,-,22/Jan/2019:12:38:27 +0330,GET,/image/29314?name=%D8%AF%DB%8C%D8%A8%D8%A7-7.j...,200,1105,https://www.zanbil.ir/product/29314/%DA%A9%D8%...,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....
1,37.152.163.59,-,22/Jan/2019:12:38:27 +0330,GET,/static/images/zanbil-kharid.png,200,358,https://www.zanbil.ir/product/29314/%DA%A9%D8%...,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....
2,85.9.73.119,-,22/Jan/2019:12:38:27 +0330,GET,/static/images/next.png,200,3045,https://znbl.ir/static/bundle-bundle_site_head...,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...
3,37.152.163.59,-,22/Jan/2019:12:38:27 +0330,GET,/image/29314?name=%D8%AF%DB%8C%D8%A8%D8%A7-4.j...,200,1457,https://www.zanbil.ir/product/29314/%DA%A9%D8%...,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....
4,85.9.73.119,-,22/Jan/2019:12:38:27 +0330,GET,/static/images/checked.png,200,1083,https://znbl.ir/static/bundle-bundle_site_head...,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...
...,...,...,...,...,...,...,...,...,...
10364860,86.104.110.254,-,26/Jan/2019:16:01:31 +0330,GET,/settings/logo,200,4120,https://www.zanbil.ir/m/browse/tv/%D8%AA%D9%84...,Mozilla/5.0 (iPhone; CPU iPhone OS 12_1 like M...
10364861,5.125.254.169,-,26/Jan/2019:16:01:31 +0330,GET,/image/5/brand,200,2171,https://www.zanbil.ir/m/filter/p62%2Cstexists,Mozilla/5.0 (iPhone; CPU iPhone OS 12_1_2 like...
10364862,65.49.68.192,-,26/Jan/2019:16:01:31 +0330,GET,/image/64646/productModel/150x150,200,5318,https://www.zanbil.ir/browse/audio-and-video-e...,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:6...
10364863,5.125.254.169,-,26/Jan/2019:16:01:31 +0330,GET,/image/1/brand,200,3924,https://www.zanbil.ir/m/filter/p62%2Cstexists,Mozilla/5.0 (iPhone; CPU iPhone OS 12_1_2 like...
